https://colab.research.google.com/github/TavernAI/TavernAI/blob/main/colab/GPU.ipynb<br>

Works with:<br>
KoboldAI https://github.com/KoboldAI/KoboldAI-Client<br>
Pygmalion https://huggingface.co/PygmalionAI/<br>
<br>
**Links**<br>
TavernAI Github https://github.com/TavernAI/TavernAI<br>
TavernAI Discord https://discord.gg/zmK2gmr45t<br>
TavernAI Boosty https://boosty.to/tavernai
<pre>
 Tavern.AI/ \ /  ^   ^ ^ ^    ~~~~ ^ \     /  ^ ^   ^ ^/ ^  ^ \/^  ^    \
         /^ ^\ ^  ^ ^   ^ ^  ~~   ^   \   /  ^  ^ ^   / ^ ^  ^/   ^ ^    \
        /^ ^ ^\^   ^ ^ ^   _||____   ^ \ /  ^  ^ ^   /       /  ^  ^  ^   \
 /\ /\ /\   ^  \  /\ /\   /\\\\\\\\   ^ \  ^ /\ /\ /\   /\ /\ /\  ^ ^  ^/\
//\\/\\/\\   ^  \//\\/\\ /__\\\\\\\\  _, \  //\\/\\/\\ //\\/\\/\\  ^ ^ //\\
//\\/\\/\\       //\\/\\ |__|_|_|__|   \__, //\\/\\/\\ //\\/\\/\\     ///\\\
 || || (@＾◡＾)(≖ ‸ ≖*) ( ←_← )\| /|   /\ \ヽ(°ㅂ°╬) |( Ψ▼ｰ▼)∈ (O_O; )  |||
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~  ~~~~~ ~~ 
</pre>
**Launch Instructions**<br>
1. Click the launch button.
2. Wait for the environment and model to load
3. After initialization, a TavernAI link will appear

**Faq**<br>
* Q: I do not get a TavernAI link
* A: It seems the localtunnel service is currently down, so the TavernAI link is unavailable. Need to wait for it to start working again.

In [ ]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }
#Taken from KoboldAI colab
%%html
<b>Press play on the music player to keep the tab alive, then start TavernAI below (Uses only 13MB of data)</b><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title <b>TavernAI</b>
#@markdown <- Click For Start (≖ ‸ ≖ ✿)

Model = "Pygmalion 6B" #@param [ "Pygmalion 6B", "Pygmalion 6B Dev"] {allow-input: true}
Version = "Official" 
Provider = "Cloudflare" 
use_google_drive = True #@param {type:"boolean"}

!nvidia-smi
import subprocess
import time
import sys
import os
import threading
from google.colab import drive
def copy_chats():
  for dirpath, dirnames, filenames in os.walk("/content/drive/MyDrive/TavernAI/chats/"):
      for filename in filenames:
        source_file = os.path.join(dirpath, filename)
        destination_file = source_file.replace("/content/drive/MyDrive/TavernAI/chats/", "/TavernAIColab/public/chats/")

        if not os.path.exists(destination_file):
          if not os.path.exists(os.path.dirname(destination_file)):
            os.makedirs(os.path.dirname(destination_file))
          !cp -r {source_file} {destination_file}
        else:
          !cp -r {source_file} {destination_file}
      
def copy_characters():
  source_files2 = os.listdir("/content/drive/MyDrive/TavernAI/characters")
  destination_files2 = os.listdir("/TavernAIColab/public/characters/")

  new_files2 = set(source_files2) - set(destination_files2)
  deleted_files2 = set(destination_files2) - set(source_files2)
  updated_files2 = [file for file in source_files2 if file in destination_files2 and 
                    os.path.getmtime(f'/content/drive/MyDrive/TavernAI/characters/{file}') > 
                    os.path.getmtime(f'/TavernAIColab/public/characters/{file}')]

  if new_files2:
    for file in new_files2:
      if os.path.isdir(f'/content/drive/MyDrive/TavernAI/characters/{file}'):
        !cp -r /content/drive/MyDrive/TavernAI/characters/{file} /TavernAIColab/public/characters/
      else:
        !cp /content/drive/MyDrive/TavernAI/characters/{file} /TavernAIColab/public/characters/

  if updated_files2:
    for file in updated_files2:
      !cp /content/drive/MyDrive/TavernAI/characters/{file} /TavernAIColab/public/characters/
      
def sync_files():
  while True:
    #chats
    for dirpath, dirnames, filenames in os.walk("/TavernAIColab/public/chats/"):
      for filename in filenames:
        source_file = os.path.join(dirpath, filename)
        destination_file = source_file.replace("/TavernAIColab/public/chats/", "/content/drive/MyDrive/TavernAI/chats/")

        if not os.path.exists(destination_file):
          if not os.path.exists(os.path.dirname(destination_file)):
            os.makedirs(os.path.dirname(destination_file))
          !cp -r {source_file} {destination_file}
        elif os.path.getmtime(source_file) > os.path.getmtime(destination_file):
          !cp -r {source_file} {destination_file}

    source_files2 = os.listdir("/TavernAIColab/public/chats/")
    destination_files2 = os.listdir("/content/drive/MyDrive/TavernAI/chats")

    deleted_files = set(destination_files2) - set(source_files2)
    if deleted_files:
      for file in deleted_files:
        if os.path.isdir(f'/content/drive/MyDrive/TavernAI/chats/{file}'):
          !rm -r /content/drive/MyDrive/TavernAI/chats/{file}
        else:
          !rm /content/drive/MyDrive/TavernAI/chats/{file}


    #characters
    source_files2 = os.listdir("/TavernAIColab/public/characters/")
    destination_files2 = os.listdir("/content/drive/MyDrive/TavernAI/characters")

    new_files2 = set(source_files2) - set(destination_files2)
    deleted_files2 = set(destination_files2) - set(source_files2)
    updated_files2 = [file for file in source_files2 if file in destination_files2 and 
                     os.path.getmtime(f'/TavernAIColab/public/characters/{file}') > 
                     os.path.getmtime(f'/content/drive/MyDrive/TavernAI/characters/{file}')]

    if new_files2:
      for file in new_files2:
        if os.path.isdir(f'/taicolabtest/public/characters/{file}'):
          !cp -r /TavernAIColab/public/characters/{file} /content/drive/MyDrive/TavernAI/characters
        else:
          !cp /TavernAIColab/public/characters/{file} /content/drive/MyDrive/TavernAI/characters

    if deleted_files2:
      for file in deleted_files2:
        !rm /content/drive/MyDrive/TavernAI/characters/{file}

    if updated_files2:
      for file in updated_files2:
        !cp /TavernAIColab/public/characters/{file} /content/drive/MyDrive/TavernAI/characters
    time.sleep(3) 
    
if use_google_drive:
  drive.mount('/content/drive/')
  if not os.path.exists("/content/drive/MyDrive/TavernAI/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/characters/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/characters/")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/chats/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/chats/")
else:
  if not os.path.exists("/content/drive"):
    os.mkdir("/content/drive")
  if not os.path.exists("/content/drive/MyDrive/"):
    os.mkdir("/content/drive/MyDrive/")

Revision = ""

if Model == "Pygmalion 6B":
  Model = "PygmalionAI/pygmalion-6b"
  path = ""
  download = ""
  Version = "United"
elif Model == "Pygmalion 6B Dev":
  Model = "PygmalionAI/pygmalion-6b"
  Revision = "--revision dev"
  path = ""
  Version = "United"
  download = ""

if Provider == "Localtunnel":
  tunnel = "--localtunnel yes"
else:
  tunnel = ""


#Henk's KoboldAI script
!wget https://koboldai.org/ckds && chmod +x ckds
!./ckds --init only
p = subprocess.Popen(['/content/ckds', '--model', Model], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

#Do not repeat! Tricks performed by a professional!
url = ''
while True:
    line = p.stdout.readline().decode().strip()
    if "KoboldAI has finished loading and is available at the following link: " in line:
        print(line)
        url = line.split("KoboldAI has finished loading and is available at the following link: ")[1]
        print(url)
        break
    if "KoboldAI has finished loading and is available at the following link for UI 1: " in line:
        print(line)
        url = line.split("KoboldAI has finished loading and is available at the following link for UI 1: ")[1]
        print(url)
        break
    if not line:
        break
    print(line)
    if "INIT" in line and "Transformers" in line:
      print("Model loading... (It will take 2 - 5 minutes)")



#TavernAI
%cd /
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.37.2/install.sh | bash
!nvm install 19.1.0
!nvm use 19.1.0
!node -v
!git clone https://github.com/TavernAI/TavernAIColab
%cd TavernAIColab
!npm install
time.sleep(1)
%env colab=2
%env colaburl=$url
!nohup node server.js &
time.sleep(3)
%cd /
copy_characters()
time.sleep(1)
copy_chats()
time.sleep(1)
if use_google_drive:
  t = threading.Thread(target=sync_files)
  t.daemon = True  # Daemonize
  t.start()
print('KoboldAI LINK:')
print(url)
print('')
print('###TavernAI LINK###')
!lt --port 8000
